In [ ]:
import sys
import json
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

# from scipy.sparse.csr import csr_matrix
from scipy.sparse import csr_matrix
from numpy import linalg as LA

# input data files
file_train = "./Filtered_data/train_sample_augmented.json"
#file_train = "./train_data_en_pred_filtered.json"
file_valid = "./Filtered_data/valid_sample_all.json"
file_test = "./Filtered_data/test_sample_all.json"
#file_valid = "./valid_sample_all_without_summer.json"
#file_test = "./test_sample_all_without_summer.json"
#file_train_unique = './train_data_all.json'

In [ ]:
test_data = pd.read_json(file_test, orient='records', lines=True)
valid_data = pd.read_json(file_valid, orient='records', lines=True)
train_data = pd.read_json(file_train, orient='records', lines=True)
#train_data_unique = pd.read_json(file_train_unique, orient='records', lines= True)

temp = test_data.head()
# print(test_data.head(), valid_data.head(), train_data.head())
print("te:", len(test_data), ",", len(set(test_data.last_semester)), "sems,", set(test_data.last_semester))
print("val:", len(valid_data), ",", len(set(valid_data.last_semester)), "sems,", set(valid_data.last_semester))
print("tr:", len(train_data), ",", len(set(train_data.last_semester)),"sems,", set(train_data.last_semester))

In [ ]:
valid_data.loc[0].baskets  # -> accessing a sequence of baskets

In [ ]:
def get_course_info(pddata, cdict, clist, offerings, ccounts):
    # update the cdict, clist, offerings, ccounts with the courses present in the pddata
    # cdict: key:courseID, value: course index, cdict[courseID] = index
    # clist: list with clist[index] = courseID
    # offerings: dict with key: semester, value: list of courses offered 
    # ccounts: dict with key: courseID, value: 
    trainflag = True
    if len(ccounts) == 1:
        trainflag = False
    for i in range(len(pddata)):
        sbaskets = pddata.loc[i].baskets
        for basket in sbaskets:
            for c in basket:
                if c not in cdict and trainflag: # only keep courses that appear in training set
                    cdict[c]=len(cdict)
                    clist.append(c)
                    ccounts[c] = 0
                if trainflag:
                    ccounts[c] += 1
        last_sem = pddata.loc[i].last_semester
        if last_sem not in offerings:
            offerings[last_sem] = []
        for c in sbaskets[-1]:
            if c not in offerings[last_sem]:
                offerings[last_sem].append(c)
    reversed_item_dict = dict(zip(cdict.values(), cdict.keys()))
    return cdict, clist, offerings, ccounts, reversed_item_dict
cdict, clist, offerings, ccounts, reversed_item_dict = get_course_info(train_data, {}, [], {}, {})
cdict, clist, offerings, _, _ = get_course_info(valid_data, cdict, clist, offerings, {1:"1"})
cdict, clist, offerings, _, _ = get_course_info(test_data, cdict, clist, offerings, {1:"1"})

In [ ]:
len(cdict), len(clist), len(offerings), len(ccounts)

In [ ]:
# plt.hist(ccounts.values(), bins=[0,3,5,10,20,50,100,900])
# plt.show() 

In [ ]:
def create_markov_chain_mat(pddata, cdict):
    nc = len(cdict)
    thematr1 = np.zeros((nc, nc))
    for i in range(len(pddata)):
        sbaskets = pddata.loc[i].baskets
        for i in range(len(sbaskets)-1):
            for c1 in sbaskets[i]:
                for c2 in sbaskets[i+1]:
                    if c1 in cdict and c2 in cdict:
                        thematr1[cdict[c1]][cdict[c2]] += 1

    return thematr1
def check_mat(thematr):
    # check correctness of thematr
    # returns the number of rows and columns that add up to 0 in the initial matrix.
    thematr = np.array(thematr)
    rows = thematr.sum(axis = 1)
    cols = thematr.sum(axis = 0)
    n = len(rows)
    rc, cc = 0, 0
    for i in range(n):
        if rows[i]==0:
            rc += 1
            thematr[i] = np.ones(n)
            # thematr[i][i] = 1
        if cols[i]==0:
            cc += 1
            for ii in range(n):
                thematr[ii][i]=1
    print(True in (thematr.sum(axis=1)==0), True in (thematr.sum(axis=0)==0))
    return (thematr, rc,cc)
def row_center(thematr):
    nc = len(thematr)
    G = np.matrix(thematr)
    G = G/G.sum(1)
    # print(G.sum(0))
    # print(G.sum(1))
    # print(G.shape, type(G))
    G = csr_matrix(G)
    return G
G1 = create_markov_chain_mat(train_data, cdict)
(G1, temp, temp1) = check_mat(G1)
print("Zero rows:",temp,"columns:",temp1,"sum(#training pairs):",np.sum(G1))
G1 = row_center(G1)

In [ ]:
import math
# calculating term dictionary where key = semester and value = another dictionary with course and nummber of actual enrolments
def calculate_term_dict(term_dict, semester, basket, reversed_item_dict):
    for item in basket:
        if semester not in term_dict:
            count_course = {}
        else:
            count_course = term_dict[semester]
        if reversed_item_dict[item] not in count_course:
            count_course[reversed_item_dict[item]] = 1
        else:
            count_course[reversed_item_dict[item]] = count_course[reversed_item_dict[item]] + 1
        term_dict[semester] = count_course
    return term_dict

def calculate_term_dict_2(term_dict, semester, basket, reversed_item_dict):
    for item in basket:
        if semester not in term_dict:
            count_course = {}
        else:
            count_course = term_dict[semester]
        if item not in count_course:
            count_course[item] = 1
        else:
            count_course[item] = count_course[item] + 1
        term_dict[semester] = count_course
    return term_dict
# calculating term dictionary where key = semester and value = another dictionary with course and nummber of predicted enrolments that are relevant (true)
def calculate_term_dict_true(term_dict_true, semester, t_basket, pred_basket, reversed_item_dict):
    for item in pred_basket:
        if item in t_basket:
            if semester not in term_dict_true:
                count_course = {}
            else:
                count_course = term_dict_true[semester]
            if item not in count_course:
                count_course[item] = 1
            else:
                count_course[item] = count_course[item]+ 1
            term_dict_true[semester] = count_course
    return term_dict_true
# calculating term dictionary where key = semester and value = another dictionary with course and nummber of predicted enrolments that are irrelevant (false)
def calculate_term_dict_false(term_dict_false, semester, t_basket, pred_basket, reversed_item_dict):
    for item in pred_basket:
        if item not in t_basket:
            if semester not in term_dict_false:
                count_course = {}
            else:
                count_course = term_dict_false[semester]
            if item not in count_course:
                count_course[item] = 1
            else:
                count_course[item] = count_course[item] + 1
            term_dict_false[semester] = count_course
    return term_dict_false
# def course_allocation(data, reversed_item_dict):
#     term_dict = {}
#     count_course = {}
#     for x in range(len(data)):
#         semester = data['last_semester'][x]
#         #if semester not in term_dict:
#         term_dict, count_course = calculate_term_dict(term_dict, semester, count_course, data['baskets'][x], reversed_item_dict)
#         # else:
#         #     term_dict[semester], count_course = calculate_term_dict(term_dict, semester, count_course, data['baskets'][x], reversed_item_dict)
#     return term_dict
# calculate average frequency of courses over semesters
def calculate_avg_n_actual_courses(input_data, reversed_item_dict):
    data = input_data
    frequency_of_courses = {}
    for baskets in data["baskets"]:
        for basket in baskets:
            for item in basket:
                if item not in frequency_of_courses:
                    frequency_of_courses[item] = 1
                else:
                    frequency_of_courses[item] += 1
    term_dict_all = {}
    for x in range(len(data)):
        baskets = data['baskets'][x]
        ts = data['timestamps'][x]
        #index1 =0 
        for x1 in range(len(ts)):
            basket = baskets[x1]
            semester = ts[x1]
            term_dict_all = calculate_term_dict_2(term_dict_all, semester, basket, reversed_item_dict)
    count_course_all = {}
    for keys, values in term_dict_all.items():
        count_course = values
        for item, cnt in count_course.items():
            if item not in count_course_all:
                count_course_all[item] = [cnt, 1]
            else:
                # list1 = count_course_all[item]
                # list1[0] = list1[0]+ cnt
                # list1[1] = list1[0]+ 1
                cnt1, n1 = count_course_all[item]
                cnt1 += cnt
                n1 += 1
                #count_course_all[item] = list1
                count_course_all[item] = [cnt1, n1]
    count_course_avg = {}
    for course, n in count_course_all.items():
        #count_course_avg[course] = float(n[0]/n[1])
        cnt2, n2 = n
        count_course_avg[course] = float(cnt2/n2)
    #calculate standard deviation
    course_sd = {}
    for keys, values in term_dict_all.items():
        count_course = values
        for item, cnt in count_course.items():
            if item not in course_sd:
                course_sd[item] = [pow((cnt-count_course_avg[item]),2), 1]
            else:
                # list1 = count_course_all[item]
                # list1[0] = list1[0]+ cnt
                # list1[1] = list1[0]+ 1
                cnt1, n1 = course_sd[item]
                cnt1 = cnt1+ pow((cnt-count_course_avg[item]),2)
                n1 += 1
                #count_course_all[item] = list1
                course_sd[item] = [cnt1, n1]
    course_sd_main = {}
    course_number_terms = {}
    for course, n in course_sd.items():
        #count_course_avg[course] = float(n[0]/n[1])
        cnt2, n2 = n
        if(n2==1): course_sd_main[course] = float(math.sqrt(cnt2/n2))
        else: course_sd_main[course] = float(math.sqrt(cnt2/(n2-1)))
        course_number_terms[course] = n2
    
    return term_dict_all, frequency_of_courses, count_course_avg, course_sd_main, course_number_terms

def find_prior_term(course, prior_semester, term_dict_all_prior):
    flag = 0
    count_course_prior_2 = {}
    while(flag!=1):
        #print("prior_semester: ", prior_semester)
        if prior_semester in term_dict_all_prior:
            count_course_prior_2 = term_dict_all_prior[prior_semester]
        if course in count_course_prior_2:
            flag =1
        if prior_semester %5==0:
            prior_semester = prior_semester-4
        else:
            prior_semester = prior_semester-3
    return count_course_prior_2 
# calculate standard deviation
def calculate_std_dev(error_list):
    sum_err = 0.0
    for err in error_list:
        sum_err += err
    avg_err = sum_err/ len(error_list)
    sum_diff = 0.0
    for err in error_list:
        sum_diff += pow((err-avg_err), 2)
    std_dev = math.sqrt((sum_diff/len(error_list)))
    return avg_err, std_dev

# checking if a course is CIS departmental course or not using acronyms
def course_CIS_dept_filtering(course):
    list_of_terms = ["CAP", "CDA", "CEN", "CGS", "CIS", "CNT", "COP", "COT", "CTS", "IDC","IDS"]
    flag = 0
    for term in list_of_terms:
        if course.find(term)!= -1:
            flag = 1
    return flag 

# calculate mse, rmse, mae
def calculate_mse_for_course_allocation(term_dict, term_dict_predicted, term_dict_predicted_true, term_dict_predicted_false, count_total_course, count_course_avg, course_sd_main, course_number_terms, term_dict_all_prior):
    mse_for_course_allocation = 0.0
    mse_for_course_allocation_2 = 0.0
    # avg_recall_for_course_allocation = 0.0
    # avg_recall_for_course_allocation_considering_not_predicted_courses = 0.0
    #avg_mse_for_course_allocation = 0.0
    avg_mse_for_course_allocation_considering_not_predicted_courses = 0.0
    mae_for_course_allocation = 0.0
    mae_for_course_allocation_2 = 0.0
    # avg_recall_for_course_allocation = 0.0
    # avg_recall_for_course_allocation_considering_not_predicted_courses = 0.0
    #avg_mse_for_course_allocation = 0.0
    avg_mae_for_course_allocation_considering_not_predicted_courses = 0.0
    msse_for_course_allocation = 0.0
    msse_for_course_allocation_2 = 0.0
    # avg_recall_for_course_allocation = 0.0
    # avg_recall_for_course_allocation_considering_not_predicted_courses = 0.0
    #avg_mse_for_course_allocation = 0.0
    avg_msse_for_course_allocation_considering_not_predicted_courses = 0.0
    mase_for_course_allocation = 0.0
    mase_for_course_allocation_2 = 0.0
    # avg_recall_for_course_allocation = 0.0
    # avg_recall_for_course_allocation_considering_not_predicted_courses = 0.0
    #avg_mse_for_course_allocation = 0.0
    avg_mase_for_course_allocation_considering_not_predicted_courses = 0.0
    #count1= 0
    count2 = 0
    #output_path1= output_dir+ "/test_course_allocation_CIS_new.txt"
    output_path1= "./scholars_walk_test_course_allocation.txt"
    f = open(output_path1, "w") #generating text file with recommendation using filtering function
    course_allocation = []
    error_list = []
    ab_error_list = []
    st_error_list = []
    for keys in term_dict.keys():
        semester = keys
        count_course = term_dict[semester]
        # number of students in the previous offering
        if semester %5==0:
            prior_semester = semester-4
        else:
            prior_semester = semester-3

        if semester in term_dict_predicted:
            count_course_predicted = term_dict_predicted[semester]
            count_course_predicted_true = term_dict_predicted_true[semester]
            count_course_predicted_false = term_dict_predicted_false[semester]

            
            for item1 in count_course.keys():
                f.write("Semester: ")
                f.write(str(semester)+ " ")
                f.write("Course ID: ")
                f.write(str(item1)+ " ")
                count_course_prior = find_prior_term(item1, prior_semester, term_dict_all_prior)

                if item1 in count_course_predicted:
                    #mse_for_course_allocation += pow(((count_course[item1]/count_total_course[semester])-(count_course_predicted[item1]/count_total_course[semester])), 2)
                    mse_for_course_allocation += pow((count_course_predicted[item1]-count_course[item1]), 2)
                    mae_for_course_allocation += abs(count_course_predicted[item1]-count_course[item1])
                    msse_for_course_allocation += pow(abs((count_course_predicted[item1]-count_course[item1])/count_course[item1]), 2)
                    mase_for_course_allocation += abs((count_course_predicted[item1]-count_course[item1])/count_course[item1])
                    error_list.append(count_course_predicted[item1]-count_course[item1])
                    ab_error_list.append(abs(count_course_predicted[item1]-count_course[item1]))
                    st_error_list.append(abs((count_course_predicted[item1]-count_course[item1])/count_course[item1]))
                    f.write("actual: ")
                    f.write(str(count_course[item1])+ " ")
                    f.write("predicted: ")
                    f.write(str(count_course_predicted[item1]))
                    f.write("\n")
                    if item1 in count_course_predicted_true and item1 in count_course_predicted_false:
                        row = [semester, item1, count_course[item1], count_course_predicted[item1], count_course_predicted_true[item1], count_course_predicted_false[item1], count_course_avg[item1], course_sd_main[item1], course_number_terms[item1]]
                        #course_allocation.append(row)
                    elif item1 in count_course_predicted_true and item1 not in count_course_predicted_false:
                        row = [semester, item1, count_course[item1], count_course_predicted[item1], count_course_predicted_true[item1], 0, count_course_avg[item1], course_sd_main[item1], course_number_terms[item1]]
                        #course_allocation.append(row)
                    elif item1 not in count_course_predicted_true and item1 in count_course_predicted_false:
                        row = [semester, item1, count_course[item1], count_course_predicted[item1], 0, count_course_predicted_false[item1], count_course_avg[item1], course_sd_main[item1], course_number_terms[item1]]
                        #course_allocation.append(row)
                    #count1 += 1
                    if item1 in count_course_prior:
                        row.append(count_course_prior[item1])
                    else:
                        row.append(0)
                    course_allocation.append(row)
                    count2 += 1
                else:
                    mse_for_course_allocation_2 += pow((count_course[item1]-0), 2)
                    mae_for_course_allocation_2 += abs((count_course[item1]-0))
                    msse_for_course_allocation_2 += pow(((count_course[item1]-0)/count_course[item1]), 2)
                    mase_for_course_allocation_2 += abs((count_course[item1]-0)/count_course[item1])
                    error_list.append(0-count_course[item1])
                    ab_error_list.append(abs(0-count_course[item1]))
                    st_error_list.append(abs((0-count_course[item1])/count_course[item1]))
                    
                    f.write("actual: ")
                    f.write(str(count_course[item1])+ " ")
                    f.write("predicted: ")
                    f.write(str(0))
                    f.write("\n")
                    if item1 in count_course_prior:
                        row = [semester, item1, count_course[item1], 0, 0, 0, count_course_avg[item1], course_sd_main[item1], course_number_terms[item1], count_course_prior[item1]]
                    else:
                        row = [semester, item1, count_course[item1], 0, 0, 0, count_course_avg[item1], course_sd_main[item1], course_number_terms[item1], 0]
                    course_allocation.append(row)
                    count2 += 1
    #avg_mse_for_course_allocation = mse_for_course_allocation/ count1
    avg_mse_for_course_allocation_considering_not_predicted_courses = (mse_for_course_allocation+ mse_for_course_allocation_2 )/ count2
    avg_mae_for_course_allocation_considering_not_predicted_courses = (mae_for_course_allocation+ mae_for_course_allocation_2 )/ count2
    avg_msse_for_course_allocation_considering_not_predicted_courses = (msse_for_course_allocation+ msse_for_course_allocation_2 )/ count2
    avg_mase_for_course_allocation_considering_not_predicted_courses = (mase_for_course_allocation+ mase_for_course_allocation_2 )/ count2
   
    f.close()
    course_allocation_actual_predicted = pd.DataFrame(course_allocation, columns=['Semester', 'Course_ID', 'actual_n', 'predicted_n', 'predicted_n_true', 'predicted_n_false', 'avg_n_actual', 'st_dev_actual', 'number_of_terms', 'n_sts_last_offering'])
    course_allocation_actual_predicted.to_csv('./scholars_walk_course_allocation_actual_predicted_v7.csv')
    return avg_mse_for_course_allocation_considering_not_predicted_courses, avg_mae_for_course_allocation_considering_not_predicted_courses, avg_msse_for_course_allocation_considering_not_predicted_courses, avg_mase_for_course_allocation_considering_not_predicted_courses, error_list, ab_error_list, st_error_list


In [ ]:
def predict4set(G1, pddata, global_info, opt):
    # """ Given the normalized matrix G,
    # We perform random walk with starting point the initial vector of the student
    # on the current semester, i.e., with 1s in the prior courses he/she has taken
    # Return the recall.(& good & bad seperately)
    # 'results' is a 3x2 matrix(list of lists) with sum(recall) & len(recall) for:
    # - all together
    # - only good courses
    # - only bad courses.
    # This affects only the nominator. if the good/bad set is empty, then we dont
    # consider this instance at all.  """
    [c_counts, offerings, clist, cdict] = global_info

    recall, percentage, avgrank, maxrank, recallgrp = {}, {}, {}, {}, {}
    nprc, ntopn, casesmissed = 0, 0, 0
    nc = len(cdict)
    if opt == -1: # check for different values of beta
        # Parameter Beta Values to try:
        beta = np.array(range(0,160,10))/200
    else: # use a specified beta
        beta = [opt]
        timespredicted, capacity, nsts = {}, {}, {} # timespredicted - capacity
    for i in range(len(beta)):
        recall[beta[i]], percentage[beta[i]], avgrank[beta[i]], maxrank[beta[i]] = [], [], [], []
        recallgrp[beta[i]] = {1:[],2:[], 3:[]}
    recall_for_at_least_one_cor_pred = 0
    count_at_least_one_cor_pred = 0
    count_at_least_two_cor_pred, count_at_least_three_cor_pred, count_at_least_four_cor_pred, count_at_least_five_cor_pred, count_all_cor_pred  = 0, 0, 0, 0, 0
    count_actual_bsize_at_least_2, count_actual_bsize_at_least_3, count_actual_bsize_at_least_4, count_actual_bsize_at_least_5, count_actual_bsize_at_least_6 = 0, 0, 0, 0, 0
    target_basket_size = {}
    target_basket_size[1] = 0
    target_basket_size[2] = 0
    target_basket_size[3] = 0
    target_basket_size[4] = 0
    target_basket_size[5] = 0
    target_basket_size[6] = 0
    count_cor_pred = {}
    total_correct_preds = 0
    for x5 in range(1,7):
        for y5 in range(1,7):
            count_cor_pred[x5,y5] = 0
    term_dict = {}
    #count_course = {}
    term_dict_predicted = {}
    term_dict_predicted_true = {}
    term_dict_predicted_false = {}
    count_CIS_target =0
    recall_bsize = {}
    missed_bsize = {}
    retake_bsize = {}
    rec_info = []
    for i in range(len(pddata)):
        sbaskets = pddata.loc[i].baskets
        #target_semester = pddata.loc[i].last_semester
        prior_bsize = len(sbaskets)-1
        sterm = pddata.loc[i].last_semester # student's target semester
        c2remove = [item for sublist in sbaskets[:-1] for item in sublist]
        c0list = [temp for temp in sbaskets[-1] if temp in cdict] # courses in target semester
        c1list = [temp for temp in sbaskets[-2] if temp in cdict] # courses in previous semester
        prior_courses = []
        for bkt in range(len(sbaskets)-1):
            for cor in sbaskets[bkt]:
                if cor not in prior_courses:
                    prior_courses.append(cor)
        if len(c0list)==0 or len(c1list)==0:
            continue
        # create vector
        start_vec1 = np.zeros(nc)
        for ic in c1list:
            start_vec1[cdict[ic]] = 1.0
        nprc += len(c1list)      # avg prior courses 1 semester before
        if sum(start_vec1) == 0:
            print("start_vec problem")
            continue
        # normalize it
        start_vec1 = start_vec1/sum(start_vec1)
        # find number of courses to predict - according to options
        # topN = 10
        topN = len(c0list)
        # random walk
        (ret, retall, vp_new) = randomwalkREWonSemester(beta, G1, start_vec1, topN, c_counts, clist, offerings[sterm], c2remove)
        
#         topN = len(c0list) # we need to compute the recall, so we need here the courses taken by the stud
#         # print predictions  ### TODO: Here check for ties of vp vector
#         if opt != -1:
#             # print(len(retall[0]), retall[0], max(retall[0]), len(vp_new))
#             print("pred", stid, sterm, len(ret[0]), end=' ')
#             for ic in range(len(retall[0])-1,-1,-1):
#                 print(clist[retall[0][ic]], end='')
#                 print(";%1.4f;"%vp_new[retall[0][ic]], end ='')
#             print()

        ntopn += topN

        if topN>=2: count_actual_bsize_at_least_2 += 1
        if topN>=3: count_actual_bsize_at_least_3 += 1
        if topN>=4: count_actual_bsize_at_least_4 += 1
        if topN>=5: count_actual_bsize_at_least_5 += 1
        if topN>=6: count_actual_bsize_at_least_6 += 1

        if topN>=6: target_basket_size[6] += 1 
        else: target_basket_size[topN] += 1 
        actual_bsize = topN
        
        for ic in range(len(ret)):
            # count hits on predicted
            tempp = 0
            recall_temp = 0
            top_items = []
            for c2 in ret[ic]:
                top_items.append(clist[c2])
                if clist[c2] in c0list:
                    tempp += 1.0
            if (tempp>=1): count_at_least_one_cor_pred += 1 
            if (tempp>=2): count_at_least_two_cor_pred+= 1
            if (tempp>=3): count_at_least_three_cor_pred+= 1
            if (tempp>=4): count_at_least_four_cor_pred+= 1
            if (tempp>=5): count_at_least_five_cor_pred+= 1
            if (tempp==topN): count_all_cor_pred+= 1
            correct_preds = tempp
            #rel_rec = len((set(top_items) & set(target_basket2)))
            recall_temp = correct_preds/topN
            row = [actual_bsize, c0list, top_items, correct_preds, recall_temp, sterm]
            rec_info.append(row)
            total_correct_preds += correct_preds
            recall_temp = correct_preds/topN
            if prior_bsize not in recall_bsize:
                recall_bsize[prior_bsize]= [recall_temp]
            else:
                recall_bsize[prior_bsize] += [recall_temp]
            target_basket2 = c0list.copy()
            rec_basket2 = top_items.copy()
            # number of non-CIS and retake courses out of missed courses    
            n_missed = 0
            n_retake = 0
            n_non_CIS =0
            for course2 in target_basket2:
                if course2 not in rec_basket2:
                    n_missed += 1
                    if course_CIS_dept_filtering(course2)==0:
                        n_non_CIS +=1
                    if course2 in prior_courses:
                        n_retake += 1
            if n_missed>0:
                if prior_bsize not in missed_bsize:
                    missed_bsize[prior_bsize]= [n_non_CIS, n_missed]
                else:
                    x3, y3 = missed_bsize[prior_bsize]
                    x3+= n_non_CIS
                    y3 += n_missed
                    missed_bsize[prior_bsize] = [x3, y3]
                    
                if prior_bsize not in retake_bsize:
                    retake_bsize[prior_bsize]= [n_retake, n_missed]
                else:
                    x4, y4 = retake_bsize[prior_bsize]
                    x4+= n_retake
                    y4 += n_missed
                    retake_bsize[prior_bsize] = [x4, y4]
            # if n_missed>0:
            #     v3= n_non_CIS/n_missed
            #     if prior_bsize not in missed_bsize:
            #         missed_bsize[prior_bsize]= [v3]
            #     else:
            #         # x3, y3 = missed_bsize[prior_bsize]
            #         # x3+= n_non_CIS
            #         # y3 += n_missed
            #         missed_bsize[prior_bsize] += [v3]
            #     v4= n_retake/n_missed
            #     if prior_bsize not in retake_bsize:
            #         retake_bsize[prior_bsize]= [v4]
            #     else:
            #         # x4, y4 = retake_bsize[prior_bsize]
            #         # x4+= n_retake
            #         # y4 += n_missed
            #         retake_bsize[prior_bsize] += [v4]


            if (actual_bsize>=6): 
                if(correct_preds==1): count_cor_pred[6,1]+= 1
                if(correct_preds==2): count_cor_pred[6,2]+= 1
                if(correct_preds==3): count_cor_pred[6,3]+= 1
                if(correct_preds==4): count_cor_pred[6,4]+= 1
                if(correct_preds==5): count_cor_pred[6,5]+= 1
                if(correct_preds>=6): count_cor_pred[6,6]+= 1
            else:
                if(correct_preds==1): count_cor_pred[actual_bsize,1]+= 1
                if(correct_preds==2): count_cor_pred[actual_bsize,2]+= 1
                if(correct_preds==3): count_cor_pred[actual_bsize,3]+= 1
                if(correct_preds==4): count_cor_pred[actual_bsize,4]+= 1
                if(correct_preds==5): count_cor_pred[actual_bsize,5]+= 1
            term_dict = calculate_term_dict_2(term_dict, sterm, c0list, reversed_item_dict)
            #course allocation for predicted courses
            term_dict_predicted = calculate_term_dict_2(term_dict_predicted, sterm, top_items, reversed_item_dict)
            term_dict_predicted_true = calculate_term_dict_true(term_dict_predicted_true, sterm, c0list, top_items, reversed_item_dict)
            term_dict_predicted_false = calculate_term_dict_false(term_dict_predicted_false, sterm, c0list, top_items, reversed_item_dict)
            # compute average rank of relevent items
            countt, hit = 0, 0
            for c1 in c0list:
                if cdict[c1] in retall[ic]:
                    countt += 1
                    hit += len(retall[ic])-retall[ic].index(cdict[c1])-1
                    # print(c1, cdict[c1], retall.index(cdict[c1]))
                else:
                    casesmissed += 1
                    # print("Missed:",stid, c1)
            if countt>0:
                avgrank[beta[ic]].append(hit/countt)
            maxrank[beta[ic]].append(len(retall[ic]))
            # target_courses= c0list
            # target_courses_CIS = course_CIS_dept(target_courses)
            # pred_courses = top_items
            # pred_courses_CIS = course_CIS_dept(pred_courses)

            recall[beta[ic]].append(tempp/topN)
                
            if tempp>0:
                percentage[beta[ic]].append(1)
                recall_for_at_least_one_cor_pred += float(tempp/topN)
    # find best recall and return only these results
    maxrecall, bmax = 0, 0
    nexamples = 1.0*len(recall[beta[0]])
    for ib in recall:
        temprecall = sum(recall[ib])/nexamples
        if temprecall>maxrecall:
            maxrecall = temprecall
            bmax = ib
    print(bmax, options, maxrecall)
    results = [[sum(recall[bmax]), nexamples], [sum(percentage[bmax]), nexamples], [sum(avgrank[bmax]), len(avgrank[bmax])], [sum(maxrank[bmax]), len(avgrank[bmax])]]
    return (results, nprc/nexamples, ntopn/nexamples, casesmissed, bmax, count_at_least_one_cor_pred, count_at_least_two_cor_pred, count_at_least_three_cor_pred, count_at_least_four_cor_pred, count_at_least_five_cor_pred, count_all_cor_pred,  count_actual_bsize_at_least_2, count_actual_bsize_at_least_3, count_actual_bsize_at_least_4, count_actual_bsize_at_least_5, count_actual_bsize_at_least_6, recall_for_at_least_one_cor_pred, count_cor_pred, target_basket_size, total_correct_preds, term_dict, term_dict_predicted, term_dict_predicted_true, term_dict_predicted_false, count_CIS_target, recall_bsize, missed_bsize, retake_bsize, rec_info)

def randomwalkREWonSemester(betas, G1, start_vec1, topN, courses, clist, possibleCourses, priorc):
    topN_ind, all_ind = [], []
    average_popularity = np.average(list(courses.values()))
    global avgnsteps
    a1 = options[-2]
    steps = options[0]
    ve1 = start_vec1
    vold = start_vec1
    for i in range(1, steps+1):
        vold = ve1
        ve1 = a1*ve1*G1 + (1-a1)*start_vec1
        ve1 = ve1/sum(ve1)
        if LA.norm(vold - ve1) < 0.0001:
            # print("Break!", i)
            break
    vp = ve1
    avgnsteps += i

    for beta in betas:
        vp_new = vp
        for i in range(len(vp)):
            if clist[i] in courses :
                vp_new[i] = vp[i]*(courses[clist[i]])**(-beta)
            else:
                vp_new[i] = vp[i]*(average_popularity)**(-beta)
        sort_ind = sorted(range(len(vp)), key=lambda k: vp_new[k])
        # remove courses not offered
        # remove courses taken from the student in the past
        # filtering
        for i in range(len(clist)-1,-1,-1):
            if clist[i] not in possibleCourses or clist[i] in priorc:
                sort_ind.remove(i)
        topN_ind.append(sort_ind[-topN:])
        all_ind.append(sort_ind)
    return topN_ind, all_ind, vp_new

In [ ]:
# possible values for alpha:
params = [1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.4, 0.6, 0.7, 0.8, 0.85, 0.9, 0.99, 0.999]
# nsteps2do = 3
# nsteps2do = 2
nsteps2do = 1  # steps allowed to run

# Validation to get best Parameters ---------------------------------
# a1
global_info = [ccounts, offerings, clist, cdict]
options = [nsteps2do, 0, 0]  # steps2take, alpha, beta
avgnsteps = 0
recalldif = 0
# for t1 in params:
#     options[-2] = t1
#     (recall_sum, _, _, _, t4, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _) = predict4set(G1, valid_data, global_info, -1)
#     diff = recall_sum[0][0]/recall_sum[0][1]
#     if diff > recalldif:
#         recalldif = diff
#         a1best = t1
#         betabest = t4


In [ ]:
# get the best values from above
#test recall
a1best = 0.0001
betabest = 0.00
options[-2] = a1best
options[-1] = betabest

(recall_ret, npriorc, ntargc, nmissed, _, count_at_least_one_cor_pred, count_at_least_two_cor_pred, count_at_least_three_cor_pred, count_at_least_four_cor_pred, count_at_least_five_cor_pred, count_all_cor_pred,  count_actual_bsize_at_least_2, count_actual_bsize_at_least_3, count_actual_bsize_at_least_4, count_actual_bsize_at_least_5, count_actual_bsize_at_least_6,recall_for_at_least_one_cor_pred, count_cor_pred, target_basket_size, total_correct_preds, term_dict, term_dict_predicted, term_dict_predicted_true, term_dict_predicted_false, count_CIS_target, recall_bsize, missed_bsize, retake_bsize, rec_info) = predict4set(G1, test_data, global_info, betabest)


In [ ]:
recall_ret

In [ ]:
# %rel score calculation
#print(nmissed)
print(count_at_least_one_cor_pred)
percentage_of_at_least_one_cor_pred = (count_at_least_one_cor_pred/ len(test_data)) * 100
#percentage_of_at_least_one_cor_pred = (count_at_least_one_cor_pred/ count_CIS_target) * 100
print("percentage_of_at_least_one_cor_pred: ", percentage_of_at_least_one_cor_pred)
percentage_of_at_least_two_cor_pred = (count_at_least_two_cor_pred/ count_actual_bsize_at_least_2) *100
print("percentage_of_at_least_two_cor_pred: ", percentage_of_at_least_two_cor_pred)
percentage_of_at_least_three_cor_pred = (count_at_least_three_cor_pred/ count_actual_bsize_at_least_3) *100
print("percentage_of_at_least_three_cor_pred: ", percentage_of_at_least_three_cor_pred)
percentage_of_at_least_four_cor_pred = (count_at_least_four_cor_pred/ count_actual_bsize_at_least_4) * 100
print("percentage_of_at_least_four_cor_pred: ", percentage_of_at_least_four_cor_pred)
percentage_of_at_least_five_cor_pred = (count_at_least_five_cor_pred/ count_actual_bsize_at_least_5) *100
print("percentage_of_at_least_five_cor_pred: ", percentage_of_at_least_five_cor_pred)
percentage_of_all_cor_pred = (count_all_cor_pred/ len(test_data)) *100
print("percentage_of_all_cor_pred: ", percentage_of_all_cor_pred)
#calculate Recall@n for whom we generated at least one correct prediction in test data
test_recall_for_one_cor_pred = recall_for_at_least_one_cor_pred/ count_at_least_one_cor_pred
print("Recall@n for whom we generated at least one correct prediction in test data: ", test_recall_for_one_cor_pred)

for x6 in range(1,7):
        percentage_of_one_cor_pred = (count_cor_pred[x6,1]/ target_basket_size[x6]) *100
        print("percentage of_one cor pred for target basket size {}: {}".format(x6, percentage_of_one_cor_pred))
        percentage_of_two_cor_pred = (count_cor_pred[x6,2]/ target_basket_size[x6]) *100
        print("percentage of_two cor pred for target basket size {}: {}".format(x6, percentage_of_two_cor_pred))
        percentage_of_three_cor_pred = (count_cor_pred[x6,3]/ target_basket_size[x6]) *100
        print("percentage of_three cor pred for target basket size {}: {}".format(x6, percentage_of_three_cor_pred))
        percentage_of_four_cor_pred = (count_cor_pred[x6,4]/ target_basket_size[x6]) *100
        print("percentage of_four cor pred for target basket size {}: {}".format(x6, percentage_of_four_cor_pred))
        percentage_of_five_cor_pred = (count_cor_pred[x6,5]/ target_basket_size[x6]) *100
        print("percentage of_five cor pred for target basket size {}: {}".format(x6, percentage_of_five_cor_pred))
        percentage_of_at_least_six_cor_pred = (count_cor_pred[x6,6]/ target_basket_size[x6]) *100
        print("percentage of_at_least_six cor pred for target basket size {}: {}".format(x6, percentage_of_at_least_six_cor_pred))

for x7 in range(1,7):
    print("total count of target basket size of {}: {}".format(x7, target_basket_size[x7]))

for x6 in range(1,7):
        print("one cor pred for target basket size of {}: {}".format(x6, count_cor_pred[x6,1]))
        print("two cor pred for target basket size of {}: {}".format(x6, count_cor_pred[x6,2]))
        print("three cor pred for target basket size of {}: {}".format(x6, count_cor_pred[x6,3]))
        print("four cor pred for target basket size of {}: {}".format(x6, count_cor_pred[x6,4]))
        print("five cor pred for target basket size of {}: {}".format(x6, count_cor_pred[x6,5]))
        print("six or more cor pred for target basket size of {}: {}".format(x6, count_cor_pred[x6,6]))

print("total correct predictions: ", total_correct_preds)
avg_cor_rec_per_student = (total_correct_preds/ count_at_least_one_cor_pred)
print("average number of courses per student correctly recommended: ", avg_cor_rec_per_student)
# recall scores for different basket sizes
recall_bsize = dict(sorted(recall_bsize.items(), key=lambda item: item[0], reverse= False))
for k, v in recall_bsize.items():
        bsize = k
        sum1 = 0
        for r in v:
            sum1 += r
        recall = sum1/len(v)
        print("prior basket size: ", bsize)
        print("number of instances: ", len(v))
        print("recall score for test data: ", recall)
 # number of non_CIS courses out of missed courses for different number of prior semesters
missed_bsize = dict(sorted(missed_bsize.items(), key=lambda item: item[0], reverse= False))
for k, v in missed_bsize.items():
        bsize = k
        tot_non_CIS, tot_missed = v
        # per_of_non_CIS = v[0]/ v[1]
        per_of_non_CIS = (tot_non_CIS/ tot_missed) *100
        print("prior basket size: ", bsize)
        # print("number of instances: ", len(v))
        print(" percentage of non CIS courses out of missed courses for test data: ", per_of_non_CIS)
    # number of retake courses out of missed courses for different number of prior semesters
retake_bsize = dict(sorted(retake_bsize.items(), key=lambda item: item[0], reverse= False))
for k, v in retake_bsize.items():
        bsize = k
        tot_retake, tot_missed = v
        per_of_retaken_courses = (tot_retake/ tot_missed) *100
        print("prior basket size: ", bsize)
        # print("number of instances: ", len(v))
        print("percentage of retaken courses out of missed courses for test data: ", per_of_retaken_courses)
test_rec_info = pd.DataFrame(rec_info, columns=['bsize', 'target_courses', 'rec_courses', 'n_rel_rec', 'recall_score', 'target_semester'])
test_rec_info.to_json('./SW_test_rec_info_2.json', orient='records', lines=True)
test_rec_info.to_csv('./SW_test_rec_info_2.csv')
# number of non_CIS courses out of missed courses for different number of prior semesters
# missed_bsize = dict(sorted(missed_bsize.items(), key=lambda item: item[0], reverse= False))
# for k, v in missed_bsize.items():
    #     bsize = k
    #     #tot_non_CIS, tot_missed = v
    #     # per_of_non_CIS = v[0]/ v[1]
    #     sum3 = 0
    #     for r in v:
    #         sum3 += r
    #     per_of_non_CIS = (sum3/ len(v)) *100
    #     print("prior basket size: ", bsize)
    #     # print("number of instances: ", len(v))
    #     print(" percentage of non CIS courses out of missed courses for test data: ", per_of_non_CIS)
# # number of retake courses out of missed courses for different number of prior semesters
# retake_bsize = dict(sorted(retake_bsize.items(), key=lambda item: item[0], reverse= False))
# for k, v in retake_bsize.items():
    #     bsize = k
    #     # tot_retake, tot_missed = v
    #     sum4 = 0
    #     for r in v:
    #         sum4 += r
    #     per_of_retaken_courses = (sum4/ len(v)) *100
    #     print("prior basket size: ", bsize)
    #     # print("number of instances: ", len(v))
    #     print("percentage of retaken courses out of missed courses for test data: ", per_of_retaken_courses)

In [ ]:
#course allocation problem

count_total_course = {}
for keys, values in term_dict.items():
    count_course_dict = values
    count_course_dict = dict(sorted(count_course_dict.items(), key=lambda item: item[1], reverse= True))
    count3 = 0
    for cnt in count_course_dict.values():
        count3 += cnt
    count_total_course[keys] = count3
    term_dict[keys] = count_course_dict
    #sorting the courses in term dictionary based on number of occurences of courses in descending order
for keys, values in term_dict_predicted.items():
    count_course_dict = values
    count_course_dict = dict(sorted(count_course_dict.items(), key=lambda item: item[1], reverse= True))
    term_dict_predicted[keys] = count_course_dict
for keys, values in term_dict_predicted_true.items():
    count_course_dict = values
    count_course_dict = dict(sorted(count_course_dict.items(), key=lambda item: item[1], reverse= True))
    term_dict_predicted_true[keys] = count_course_dict
    
for keys, values in term_dict_predicted_false.items():
    count_course_dict = values
    count_course_dict = dict(sorted(count_course_dict.items(), key=lambda item: item[1], reverse= True))
    term_dict_predicted_false[keys] = count_course_dict

 #calculate average for total number of times a course is taken at each semester using training data 
# all_data_en_pred = pd.read_json('./all_data_en_pred_filtered.json', orient='records', lines= True)
all_data_en_pred = pd.read_json('./all_data_en_pred_filtered.json', orient='records', lines= True)

term_dict_all, frequency_of_courses, count_course_avg, course_sd_main, course_number_terms = calculate_avg_n_actual_courses(all_data_en_pred, reversed_item_dict)

# valid_data_unique = pd.read_json('./Filtered_data/valid_sample_filtered_unique.json', orient='records', lines= True)
# term_dict_valid, frequency_of_courses2, count_course_avg2, course_sd_main2, course_number_terms2 = calculate_avg_n_actual_courses(valid_data_unique, reversed_item_dict)

# avg_mse_for_course_allocation, avg_mse_for_course_allocation_considering_not_predicted_courses = calculate_mse_for_course_allocation(term_dict, term_dict_predicted)
# avg_rmse_for_course_allocation, avg_rmse_for_course_allocation_considering_not_predicted_courses = math.sqrt(avg_mse_for_course_allocation), math.sqrt(avg_mse_for_course_allocation_considering_not_predicted_courses)
avg_mse_for_course_allocation_considering_not_predicted_courses, avg_mae_for_course_allocation_considering_not_predicted_courses, avg_msse_for_course_allocation_considering_not_predicted_courses, avg_mase_for_course_allocation_considering_not_predicted_courses, error_list, ab_error_list, st_error_list = calculate_mse_for_course_allocation(term_dict, term_dict_predicted, term_dict_predicted_true, term_dict_predicted_false, count_total_course, count_course_avg, course_sd_main, course_number_terms, term_dict_all)
avg_rmse_for_course_allocation_considering_not_predicted_courses = math.sqrt(avg_mse_for_course_allocation_considering_not_predicted_courses)
avg_rmsse_for_course_allocation_considering_not_predicted_courses = math.sqrt(avg_msse_for_course_allocation_considering_not_predicted_courses)
mean_error, std_dev_error = calculate_std_dev(error_list)
mean_ab_error, std_dev_ab_error = calculate_std_dev(ab_error_list)
mean_st_error, std_dev_st_error = calculate_std_dev(st_error_list)

#print("avg mse for # of allocated course where we are predicting a course at least once: ",avg_mse_for_course_allocation)
#print("avg_mse_for_course_allocation_considering all courses available in test data: ",avg_mse_for_course_allocation_considering_not_predicted_courses)
#print("avg rmse for # of allocated course where we are predicting a course at least once: ",avg_rmse_for_course_allocation)
print("avg_mae_for_course_allocation_considering all courses available in test data: ",avg_mae_for_course_allocation_considering_not_predicted_courses)
print("avg_rmse_for_course_allocation_considering all courses available in test data: ",avg_rmse_for_course_allocation_considering_not_predicted_courses)
print("avg_mase_for_course_allocation_considering all courses available in test data: ",avg_mase_for_course_allocation_considering_not_predicted_courses)
print("avg_rmsse_for_course_allocation_considering all courses available in test data: ",avg_rmsse_for_course_allocation_considering_not_predicted_courses)
print("mean of errors: ", mean_error)
print("standard_deviation for errors: ", std_dev_error)
print("mean of absolute errors: ", mean_ab_error)
print("standard_deviation for absolute errors: ", std_dev_ab_error)
print("mean of normalized errors: ", mean_st_error)
print("standard_deviation for normalized errors: ", std_dev_st_error)

In [ ]:
#train recall
# get the best values from above
a1best = 0.0001
betabest = 0.00
options[-2] = a1best
options[-1] = betabest


(recall_ret, npriorc, ntargc, nmissed, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _) = predict4set(G1, train_data, global_info, betabest)

In [ ]:
#validation recall
# get the best values from above
a1best = 0.0001
betabest = 0.00
options[-2] = a1best
options[-1] = betabest


(recall_ret, npriorc, ntargc, nmissed, bmax, count_at_least_one_cor_pred, count_at_least_two_cor_pred, count_at_least_three_cor_pred, count_at_least_four_cor_pred, count_at_least_five_cor_pred, count_all_cor_pred,  count_actual_bsize_at_least_2, count_actual_bsize_at_least_3, count_actual_bsize_at_least_4, count_actual_bsize_at_least_5, count_actual_bsize_at_least_6, recall_for_at_least_one_cor_pred, count_cor_pred, target_basket_size, total_correct_preds, term_dict, term_dict_predicted, term_dict_predicted_true, term_dict_predicted_false, count_CIS_target, recall_bsize, missed_bsize, retake_bsize, _) = predict4set(G1, valid_data, global_info, betabest)
recall_bsize = dict(sorted(recall_bsize.items(), key=lambda item: item[0], reverse= False))
for k, v in recall_bsize.items():
        bsize = k
        sum1 = 0
        for r in v:
            sum1 += r
        recall = sum1/len(v)
        print("prior basket size: ", bsize)
        print("number of instances: ", len(v))
        print("recall score for validation data: ", recall)
# number of non_CIS courses out of missed courses for different number of prior semesters
missed_bsize = dict(sorted(missed_bsize.items(), key=lambda item: item[0], reverse= False))
for k, v in missed_bsize.items():
        bsize = k
        tot_non_CIS, tot_missed = v
        # per_of_non_CIS = v[0]/ v[1]
        per_of_non_CIS = (tot_non_CIS/ tot_missed) *100
        print("prior basket size: ", bsize)
        # print("number of instances: ", len(v))
        print(" percentage of non CIS courses out of missed courses for validation data: ", per_of_non_CIS)
# number of retake courses out of missed courses for different number of prior semesters
retake_bsize = dict(sorted(retake_bsize.items(), key=lambda item: item[0], reverse= False))
for k, v in retake_bsize.items():
        bsize = k
        tot_retake, tot_missed = v
        per_of_retaken_courses = (tot_retake/ tot_missed) *100
        print("prior basket size: ", bsize)
        # print("number of instances: ", len(v))
        print("percentage of retaken courses out of missed courses for validation data: ", per_of_retaken_courses)

# number of non_CIS courses out of missed courses for different number of prior semesters
# missed_bsize = dict(sorted(missed_bsize.items(), key=lambda item: item[0], reverse= False))
# for k, v in missed_bsize.items():
    #     bsize = k
    #     #tot_non_CIS, tot_missed = v
    #     # per_of_non_CIS = v[0]/ v[1]
    #     sum3 = 0
    #     for r in v:
    #         sum3 += r
    #     per_of_non_CIS = (sum3/ len(v)) *100
    #     print("prior basket size: ", bsize)
    #     # print("number of instances: ", len(v))
    #     print(" percentage of non CIS courses out of missed courses for test data: ", per_of_non_CIS)
# # number of retake courses out of missed courses for different number of prior semesters
# retake_bsize = dict(sorted(retake_bsize.items(), key=lambda item: item[0], reverse= False))
# for k, v in retake_bsize.items():
    #     bsize = k
    #     # tot_retake, tot_missed = v
    #     sum4 = 0
    #     for r in v:
    #         sum4 += r
    #     per_of_retaken_courses = (sum4/ len(v)) *100
    #     print("prior basket size: ", bsize)
    #     # print("number of instances: ", len(v))
    #     print("percentage of retaken courses out of missed courses for test data: ", per_of_retaken_courses)